In [3]:
from transformers import BertForQuestionAnswering, AutoTokenizer, DefaultDataCollator, TrainingArguments, Trainer, BertTokenizer
import torch

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name) # tokenizer is used for breaking down text into smaller units called tokens
model = BertForQuestionAnswering.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
example = {"key": 1, "transcript": "Heading is two six zero, target is black, white, and yellow commercial aircraft, tool to deploy is surface-to-air missiles.", "tool": "surface-to-air missiles", "heading": "260", "target": "black, white, and yellow commercial aircraft"}

context = example["transcript"]
question = "What is the heading?"

inputs = tokenizer(question, context, return_tensors='pt')
with torch.no_grad():
    outputs = model(**inputs) # ** unpacks the dictionary 'inputs' created by the tokenizer into keyword arguments for the model

# Find the tokens with the highest `start` and `end` scores
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1

# Convert tokens to answer string
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0, answer_start:answer_end]))
print("Answer:", answer)

Answer: 


In [9]:
from pathlib import Path
import jsonlines

current_directory = Path.cwd()
file_path = current_directory / '..' / '..' / 'novice' / 'nlp.jsonl'
nlp_file = file_path.resolve()
print(f"nlp_file: {nlp_file}\ncurrent_directory: {current_directory}")


# answers: obj['tool'], obj['heading'], object['target']
# context: obj['transcript']
# question: we choose a fixed qn to ask
with jsonlines.open(nlp_file) as reader:
    for obj in reader:
        print(obj)
        break
    

nlp_file: /home/jupyter/novice/nlp.jsonl
current_directory: /home/jupyter/til-24-base/nlp
{'key': 0, 'transcript': 'Heading is one five zero, target is green commercial aircraft, tool to deploy is electromagnetic pulse.', 'tool': 'electromagnetic pulse', 'heading': '150', 'target': 'green commercial aircraft'}
